# IRIS-HEP CMS Integration Challenge: Workflow Demonstration

This notebook demonstrates the workflow of the IRIS-HEP CMS integration challenge. The workflow is modular and consists of the following steps:

1. Environment Setup
2. Configuration Construction
3. Metadata Extraction
4. Skimming
5. Histogramming
6. Statistical Analysis
7. Plotting

## 1. Environment Setup

To set up the environment, use the following command to launch JupyterLab with the required dependencies:

```sh
pixi run lab
```

Alternatively, if you prefer a Conda-like environment, activate it using the provided script:

```sh
source pixi_activate.sh
```

In [ ]:
# [coffea-casa] Install packages
!pip install omegaconf
#!pip install coffea==2025.10.0
!pip install s3pathlib

In [ ]:
# Dask stuff
# Register some modules for dask workers to pickle/unpickle
import cloudpickle
import utils, user
cloudpickle.register_pickle_by_value(utils)
cloudpickle.register_pickle_by_value(user)

# Setup some funcitons for dask client handling
from dask.distributed import Client, LocalCluster, PipInstall

COFFEA_CASA = True
NEW_BUCKET = True

from dask.distributed import WorkerPlugin
import os

class EnvSetter(WorkerPlugin):
    def __init__(self, env_vars):
        self.env_vars = env_vars

    def setup(self, worker):
        import os
        os.environ.update(self.env_vars)
        print(f"Set env on worker {worker.address}: {self.env_vars}")

ID = "M4S6FZBLCE2MT5R5R3TE" if NEW_BUCKET else os.environ["AWS_ACCESS_KEY_ID"]
KEY = "1POhQ20OjZWViVuIan0bqVuegnYR8LUKmGFbpB7T" if NEW_BUCKET else os.environ["AWS_SECRET_ACCESS_KEY"]

env_setter_plugin = EnvSetter({"AWS_ACCESS_KEY_ID": ID, "AWS_SECRET_ACCESS_KEY": KEY})

def get_client(n_workers=4):
    if not COFFEA_CASA:
        cluster = LocalCluster(n_workers=n_workers, processes=True, threads_per_worker=1)
        client = Client(cluster)
        print(f"Dask dashboard: {cluster.dashboard_link}")
    else:
        dependencies = ["git+https://github.com/MoAly98/coffea.git@feat/storage_opts_parquet", "s3pathlib"]
        client = Client("tls://localhost:8786")
        client.register_plugin(PipInstall(packages=dependencies))
        client.register_plugin(env_setter_plugin)
        cluster = None  # no local cluster in this mode
    return client, cluster

def close_client(cluster, client):
    if client:
        client.close()
    if cluster:
        cluster.close()
    return None, None

## 2. Configuration Construction

A lightweight Python config defines general settings, datasets, cuts, observables, and systematics — all are type-checked and (mostly) CLI-overrideable.
Everything else in the workflow reads from it.

In [ ]:
# Import Rich-based Configuration Display from logging module
from utils.logging import display_config_table, get_config_logger

# Create a global config logger instance for this notebook
config_logger = get_config_logger()

In [ ]:
# Example: Demonstrate configuration comparison and change detection
from user.configuration import config as original_config
from utils.schema import Config, load_config_with_restricted_cli

import copy

# Save the original configuration for comparison
config = copy.deepcopy(original_config)

print("=== Full configuration ===")
display_config_table(config, expand=True)

In [ ]:
# Let's look at the datasets config
display_config_table({"datasets": config["datasets"]},
                    expand=True)

In [ ]:
# Make a modification
config["datasets"]["max_files"] = 1 # Limit to 1 files per dataset
config["general"]["processes"] = ["signal", "ttbar_semilep"]

print("=== SHOW ONLY CHANGES ===")
display_config_table({"datasets": config["datasets"]},
                    expand=True,
                    compare_with={"datasets": original_config["datasets"]},
                    show_only_changes=True)

cli_args = []  # No CLI overrides for this demo
full_config = load_config_with_restricted_cli(config, cli_args)

print("✅ Processed CLI arguments and loaded full configuration")
validated_config = Config(**full_config)  # This is the key validation step!

# 3. Output Manager & Directory Structure

The framework uses a centralized `OutputDirectoryManager` (`utils/output_manager.py`) to organize all analysis outputs.

In [ ]:
from utils.output_manager import OutputDirectoryManager

output_manager = OutputDirectoryManager(
    root_output_dir=validated_config.general.output_dir,
    cache_dir=validated_config.general.cache_dir,
    metadata_dir=validated_config.general.metadata_dir,
    skimmed_dir=validated_config.general.skimmed_dir
)

print("✅ Created OutputDirectoryManager with validated paths")


## 4. Metadata Extraction

Metadata extraction runs `coffea`'s preprocessor to build analysis workitems, a `coffea`-compatible fileset and a summary of the processed `NanoAOD`s. Its outputs stay in memory, but are also stored in `JSON` files to allow re-runing subsequent steps without pre-processing every time.

In [ ]:
from utils.metadata_extractor import NanoAODMetadataGenerator
from utils.datasets import ConfigurableDatasetManager

# Step 1: Create dataset manager instance
dataset_manager = ConfigurableDatasetManager(validated_config.datasets)

# Step 2: Create metadata generator instance
validated_config.general.run_metadata_generation=False

client, cluster = None, None #get_client()
metadata_generator = NanoAODMetadataGenerator(
    dataset_manager=dataset_manager,
    output_manager=output_manager
)
# In the back this uses coffea's preprocess function
metadata = metadata_generator.run(
    generate_metadata=validated_config.general.run_metadata_generation,
    processes_filter=validated_config.general.processes if hasattr(validated_config.general, 'processes') else None
)
#close_client(client, None)

# 5. Skimming & Event Selection

The skimming step processes the workitems from previous step, and applies event and branch selections with `dask` and a `coffea`-like processor (see [Alex's issue](https://github.com/scikit-hep/coffea/issues/1393)). Currently the skimming results are in-memory events, but also stored on disk as `ROOT` files to avoid having to run this multiple times. The events are also cached in `.pkl` files for faster re-reading. Eventually this step needs to be integrated with subsequent steps in a complete `coffea` or `coffea`-like processor.

In [ ]:
from utils.skimming import process_workitems_with_skimming

# Extract workitems and fileset from metadata generator
fileset = metadata_generator.fileset
workitems = metadata_generator.workitems

print(f"📊 Processing {len(workitems)} workitems across {len(fileset)} datasets")

# Disable caching for demonstration
validated_config.general.read_from_cache = False
validated_config.general.run_skimming = True

# Skim data with dask according to the workitems
datasets = process_workitems_with_skimming(
    workitems,
    validated_config,
    output_manager,
    datasets,
    metadata_generator.nanoaods_summary
)

# Display the structure of processed datasets
print(f"\n✅ Skimming complete! Processed datasets structure:")
for dataset in datasets:
    if dataset.events:
        print(f"  📁 {dataset.name} processed")
        for i, (events, metadata) in enumerate(dataset.events):
            print(f"    └── File {i+1}: {len(events)} events, {len(events.fields)} branches")

print(f"\n💾 Skimmed data saved to: {output_manager.get_skimmed_dir()}")

for dataset in datasets:
    print(dataset.events)

In [ ]:
# Enable caching to show how it speeds up subsequent runs
validated_config.general.read_from_cache = True
validated_config.general.run_skimming = False
# Run the skimming again - it will look for cached files
# if they don't exist, it will fallback to skim regularly
cached_datasets = process_workitems_with_skimming(
    workitems,
    validated_config,
    output_manager,
    fileset,
    metadata_generator.nanoaods_summary
)

# Show cache directory contents
import os
cache_files = os.listdir(output_manager.get_skimmed_dir())
print(f"💾 Cached files in skimmed directory: {len(cache_files)} files")
print(f"📁 Cache location: {output_manager.get_cache_dir()}")

# 6. Analysis & Histogramming

This step encapsulates a few underlying function calls:
1. Global event selection is applied
2. If MVA training is configured and enabled, an MVA training will be triggered
3. Apply corrections from `correctionlib`
4. Ghost observables will be computed and added to event record
6. Channels will be built with their corresponding selections
7. Compute all observables (once nomina, once per systematic variation)
8. Create histograms

In [ ]:
# Import the analysis class for the analysis
from analysis.nondiff import NonDiffAnalysis
from utils.output_files import save_histograms_to_root

validated_config.general.run_histogramming=True

# Initialize the analysis object
# This analysis object will handle histogram creation, observable computation, and systematic variations
nondiff_analysis = NonDiffAnalysis(validated_config, datasets, output_manager)

print(f"🔬 Analysis initialized for {len(nondiff_analysis.datasets)} datasets")

# Loop over each dataset and its associated event data
histogram_count = 0
for dataset in datasets:
    if dataset.events:
        for events, metadata in dataset.events:
            print(f"   • Processing {len(events)} events with metadata: {metadata['process']}")
            nondiff_analysis.process(events, metadata)
            histogram_count += 1

save_histograms_to_root(
    nondiff_analysis.nD_hists_per_region,
    output_file=nondiff_analysis.output_manager.get_histograms_dir() / "histograms.root",
)

print(f"📈 Generated histograms for channels: {[ch.name for ch in validated_config.channels]}")

# 7. Statistical Analysis

The statistical analysis step performs a fit from the histograms and visualize results. This uses `cabinetry` with a manually created config. It also creates and stores the corresponding `pyhf` workspace.

**Note: the following fit is performed with the full CMS open-data dataset, not the small example here** 

In [ ]:
import cabinetry
validated_config.statistics.cabinetry_config = "./example-old/outputs/cabinetry/"
cabinetry_config = cabinetry.configuration.load(
    validated_config.statistics.cabinetry_config
)
data, fit_results, pre_fit_predictions, postfit_predictions = (
    nondiff_analysis.run_fit(cabinetry_config=cabinetry_config)
)
cabinetry.visualize.data_mc(
    pre_fit_predictions,
    data,
    close_figure=False,
    config=cabinetry_config,
    figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),

)
cabinetry.visualize.data_mc(
    postfit_predictions,
    data,
    close_figure=False,
    config=cabinetry_config,
    figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),
)
cabinetry.visualize.pulls(fit_results, close_figure=False, figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),)

## 8. Full Workflow

The above workflow is broken down for demonstration, but the simplest way to use it is by putting the main pieces together in one simple steering script and running it from the command-line with CLI overrides of config options. The current implementation is steered by `analysis.py` and can be simply executed with:

```sh
python analysis.py
```

---